In [ ]:
!pip install fastapi uvicorn nest-asyncio 

In [3]:
from fastapi import FastAPI, File, UploadFile
from fastapi.staticfiles import StaticFiles
from pathlib import Path
from uuid import uuid4
from datetime import datetime

app = FastAPI()


@app.post("/upload")
async def create_upload_file(file: UploadFile = File(...)):
    print("hoge")
    # 拡張子のみ取得
    extension = Path(file.filename).suffix

    # uuid生成
    uuid = str(uuid4())
    # 日付を取得
    date = datetime.now().strftime("%Y%m%d%H%M%S")

    # ファイル名生成
    safe_filename = f"{uuid}_{date}{extension}"

    # ディレクトリが存在しなければ作成
    save_dir = Path("files")
    save_dir.mkdir(parents=True, exist_ok=True)

    # ファイル保存
    save_path = f"{save_dir}/{safe_filename}"

    with open(save_path, "wb+") as f:
        f.write(file.file.read())

    return {"filename": safe_filename}


app.mount("/", StaticFiles(directory="dist", html=True), name="static")

import nest_asyncio

nest_asyncio.apply()


from google.colab import output

output.serve_kernel_port_as_iframe(8000)


import uvicorn

uvicorn.run(app, port=8000, host="0.0.0.0")

ModuleNotFoundError: No module named 'google'